In [1]:
import pandas as pd
import numpy as np
import csv
import time
import requests as req
import re
from bs4 import BeautifulSoup as bs

In [2]:
# Function used to parse the URL
def link(url):
    header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36',
        'Cookie':'__mta=107321789.1632395680496.1633500232556.1633500307118.56; uuid_n_v=v1; _lxsdk_cuid=17c125d5ac5c8-08c88550cde595-a7d173c-1fa400-17c125d5ac5c8; __utma=17099173.1733646556.1633499883.1633499883.1633499883.1; __utmz=17099173.1633499883.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __utmb=17099173.3.10.1633499883; _csrf=59e320c4381f8964033766f06aac408a39c74cc00e6a1123b3f990a12548d0fb; _lx_utm=utm_source%3Dbing%26utm_medium%3Dorganic; uuid=18B5EF70266D11EC8BAB0F3D6466135EC1C15304F5174659BFDEDA76D77C9EE8; lt=CzzisNSdNZe6uByGW0tQKrsOuVEAAAAAvQ4AAEpMZDZBPMBmnhNmU7Gf6Q2MixzY0lWlG6mh57JQGBusHSN5-JmzcjizOyc9LP1WLQ; lt.sig=pS9wfGFfNmJPO-jzdZ1cbUm2g4I; uid=1860430993; uid.sig=aQb11UnQXxDf-xJkV7C9IRDVa3o; _lxsdk=18B5EF70266D11EC8BAB0F3D6466135EC1C15304F5174659BFDEDA76D77C9EE8; Hm_lvt_703e94591e87be68cc8da0da7cbd0be2=1633499726,1633499730,1633501037,1633501055; __mta=107321789.1632395680496.1633501056476.1633501221713.84; Hm_lpvt_703e94591e87be68cc8da0da7cbd0be2=1633501224; _lxsdk_s=17c53de54d4-e45-004-168%7C%7C183'}
    res = req.get(url,headers = header)
    if res.status_code == 200:
        return bs(res.text,"html.parser")
    return None

In [3]:
# Define the lists to store the data
ranks=[]
titles=[]
links=[]
ratings=[]
directors=[]
actors=[]
types=[]
releaseTimes=[]
durations=[]
firstWeekIncomes=[]
totalIncomes=[]
countries=[]
print("Start crawling!")
for i in range(0,100,10):
    url='https://maoyan.com/board/4?offset='+str(i) #The offset of each page is 10
    movies = link(url).find_all("dd")

    for i in movies:
        ranks.append(i.find("i").text) 
        titles.append(i.find("a").get("title"))
        ratings.append(i.find("i",class_="integer").text+i.find("i",class_="fraction").text) 
        releasetime=re.findall("上映时间：(.*)",i.find("p",class_ = "releasetime").text)[0]
        releaseTimes.append(re.sub(u"\\(.*?\\)|\\{.*?}|\\[.*?]", "", releasetime))
        actors.append(re.findall("主演：(.*)",i.find("p",class_ = "star").text)[0])
        inner_url = "https://maoyan.com" +i.find("p",class_="name").a.get("href")
        links.append(inner_url)
        inner_page= link(inner_url)
        print("Currently crawling the:  ",inner_url)
        time.sleep(1)
        directors.append(inner_page.find("a",class_="name").text.replace("\n","").replace(" ",""))
        if len(inner_page.findAll(class_='mbox-name'))!=0:
            firstWeekIncomes.append(inner_page.findAll(class_='mbox-name')[0].text)
            totalIncomes.append(inner_page.findAll(class_='mbox-name')[2].text)
        else:
            firstWeekIncomes.append('暂无')
            totalIncomes.append('暂无')
        flag=0
        temp_str=""
        for t in inner_page.findAll(class_='text-link'):
            if flag == 0:
                temp_str += str(t.text)
                flag = 1
            else:
                temp_str += "," + str(t.text) 
        types.append(temp_str)
        countriesAndDuration = inner_page.findAll("li", class_='ellipsis')[1].text
        countriesAndDuration = countriesAndDuration.strip().replace('\n', '').replace(' ', '')
        countriesAndDurationList = countriesAndDuration.split('/')
        countries.append(countriesAndDurationList[0]) 
        durations.append(countriesAndDurationList[1].strip("分钟"))
print("Crawing finished!")

Start crawling!
Currently crawling the:   https://maoyan.com/films/1200486
Currently crawling the:   https://maoyan.com/films/1297
Currently crawling the:   https://maoyan.com/films/1206605
Currently crawling the:   https://maoyan.com/films/1292
Currently crawling the:   https://maoyan.com/films/1211270
Currently crawling the:   https://maoyan.com/films/1203
Currently crawling the:   https://maoyan.com/films/1216365
Currently crawling the:   https://maoyan.com/films/1303
Currently crawling the:   https://maoyan.com/films/4055
Currently crawling the:   https://maoyan.com/films/46818
Currently crawling the:   https://maoyan.com/films/416
Currently crawling the:   https://maoyan.com/films/1633
Currently crawling the:   https://maoyan.com/films/1212
Currently crawling the:   https://maoyan.com/films/78341
Currently crawling the:   https://maoyan.com/films/1304
Currently crawling the:   https://maoyan.com/films/70976
Currently crawling the:   https://maoyan.com/films/3667
Currently crawling

In [4]:
# Save the data into the local file

df=pd.DataFrame({'Rank':ranks,
                    'Title':titles,
                    'Rating':ratings,
                    'Director':directors,
                    'Actor':actors,
                    'Countries':countries,
                    'Type':types,
                    'ReleaseTime':releaseTimes,
                    'Duration':durations,
                    'FirstWeekIncome':firstWeekIncomes,
                    'TotalIncome':totalIncomes,
                    'Link':links})

df.to_csv('MaoYanTop100.csv',encoding='utf-8-sig',index=False) #You can name it what you like
print("Save file successfully!")

Save file successfully!
